We should implement and back-test trading algorithms using historical market data.

The project is divided into three main strategies:

1. Moving Average and Momentum Strategies,
2. Value-Based Strategies, and
3. Sentiment-Based Strategies.

Write our own Back Testing Code

▪ Back Testing simulates how a trading strategy would have performed in the past using historical data.
▪ The purpose is to evaluate the strategy's effectiveness, identify potential issues, and refine it before deploying it in live trading.
▪ Here are key components a back-testing code needs to handle:
    1. Load up and process price and other data
    2. Clean and prepare data
    3. Implement logic to buy and sell based on signals
    4. Define trades and measure their performance over time
    5. Incorporate realistic transaction costs
    6. Calculate metrics – return, drawdown, Sharpe ratio etc.…
    7. Visualise results

Validation and Sanity Checks

▪ Out-of-Sample Testing:
    ▪ After optimizing the strategy on historical data, test it on a separate dataset (out-of-sample data) to verify its robustness.
▪ Sanity Checks:
    ▪ Ensure the backtest is realistic (e.g., no future data leakage, no unrealistic execution assumptions) to prevent overestimating the strategy’s performance.

In [2]:
import pandas as pd 
import numpy as np 
import os

# Define the directory containing CSV files
csv_directory = "sp500/csv"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(csv_directory) if f.endswith(".csv")]

# Initialize an empty list to store DataFrames
dfs = []

# Iterate through CSV files
for file in csv_files:
    file_path = os.path.join(csv_directory, file)
    
    # Read CSV into DataFrame
    df = pd.read_csv(file_path, usecols=["Date", "Adjusted Close"])
    
    # Rename "Adjusted Close" column to "filename_adjclose"
    stock_name = os.path.splitext(file)[0]  # Extract filename without .csv
    df.rename(columns={"Adjusted Close": f"{stock_name}_adjclose"}, inplace=True)
    
    # Append to list
    dfs.append(df)

# Merge all DataFrames on "Date"
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on="Date", how="outer")  # Outer join to retain all dates

# Conver to date 

merged_df["Date"] = pd.to_datetime(merged_df["Date"], format="%d-%m-%Y")

# Sort by date
merged_df.sort_values(by="Date", inplace=True)

Strategy 1: Moving Average and Momentum Strategies

Rules based Moving average startegy

Understand & implement moving average strategies
    ▪ Simple Moving Average (SMA)
    ▪ Write code that calculates this to different periods

Tasks:
    1. Implement a strategy where a short-term moving average (e.g., S-day SMA) crosses above or below a long-term moving average (e.g., L-day SMA).
    2. Write code to execute buy orders when the short-term average crosses above the long-term average and sell orders when the opposite occurs.
    3. Test the algorithm on a broad range of stocks (at least 100) from the S&P index for a range of values of S and L
    4. You should report average P&L and variance of P&L for each combination of moving average periods

In [ ]:
def implement_strategy(df, S, L):
    df_copy = df.copy()

    all_avg_pnl = []  # Initialize empty list
    all_var_pnl = []  # Initialize empty list

    # Get all stock columns (excluding 'Date')
    stock_columns = df_copy.columns.drop('Date')

    for stock in stock_columns:
        # Calculate moving averages
        df_copy[f'{stock}_S'] = df_copy[stock].rolling(window=S).mean()
        df_copy[f'{stock}_L'] = df_copy[stock].rolling(window=L).mean()
        
        # Generate signals
        signals = pd.Series(0, index=df_copy.index)
        signals[df_copy[f'{stock}_S'] > df_copy[f'{stock}_L']] = 1
        signals[df_copy[f'{stock}_S'] < df_copy[f'{stock}_L']] = -1
        signals = signals.shift(1).fillna(0)  # Shift signals by one day

        # Simulate trades
        position = 0
        positions = pd.Series(0, index=df_copy.index)
        for i, signal in signals.items():
            if signal == 1:
                position = 1
            elif signal == -1:
                position = 0
            positions[i] = position

        # Calculate returns
        daily_returns = df_copy[stock].pct_change() * positions
        pnl = daily_returns.sum()

        # Only append non-NaN values
        if not np.isnan(pnl):
            all_avg_pnl.append(pnl)
            all_var_pnl.append(daily_returns.var())  # Variance of daily returns

    # Calculate average P&L and variance with error handling
    avg_pnl = np.nanmean(all_avg_pnl) if len(all_avg_pnl) > 0 else 0
    var_pnl = np.nanmean(all_var_pnl) if len(all_var_pnl) > 0 else 0

    return {'avg_pnl': avg_pnl, 'var_pnl': var_pnl}

# Backtesting
S_values = [5, 10, 20]
L_values = [20, 50, 100,200]

results = []  # Initialize empty list

# Test on a subset of stocks (e.g., first 100)
subset_df = merged_df.iloc[:, :101]  # Adjust as needed

for S in S_values:
    for L in L_values:
        if S < L:  # Ensure short-term MA is less than long-term MA
            result = implement_strategy(subset_df, S, L)
            results.append({'S': S, 'L': L, 'avg_pnl': result['avg_pnl'], 'var_pnl': result['var_pnl']})

# Report results
results_df = pd.DataFrame(results)
print(results_df.to_markdown(index=False, numalign="left", stralign="left"))



C:\Users\DELL\AppData\Local\Temp\ipykernel_11216\1336796075.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy[f'{stock}_S'] = df_copy[stock].rolling(window=S).mean()
C:\Users\DELL\AppData\Local\Temp\ipykernel_11216\1336796075.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_copy[f'{stock}_L'] = df_copy[stock].rolling(window=L).mean()
C:\Users\DELL\AppData\Local\Temp\ipykernel_11216\1336796075.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many ti

PermissionError: [Errno 13] Permission denied

Rules based Momentum startegy

Understand the concept of momentum
    ▪ Relative Strength Index (RSI)
    ▪ Write code to calculate these

Tasks:
    1. Implement a strategy that buys assets when momentum indicators signal strength (e.g., RSI > 70) and sells when they signal weakness (e.g., RSI < 30).
    2. Combine momentum signals with moving averages to enhance the strategy.
    3. Test the algorithm on a broad range of stocks (at least 100) from the S&P index
    4. You should report average P&L and variance of P&L for each combination of moving average periods
    5. Once again try five best combinations of RSI you can find both alone and combined with moving averages

ML based Moving average strategy

Tasks:
    1. Build a Deep Learning model that takes in the stock prices and ML indicators and RSI indicators as features
    2. Use a 3-layer neural network (1 hidden layers) where the inputs are the indicators and the output is a buy, sell or hold signal
    3. You train it over a subset of the time series
    4. You test it on another part of the timeseries
    5. See if the changing number of layers or neurons per layer helps
    6. Test the algorithm on a broad range of stocks (at least 100) from the S&P index
    7. You should report average P&L and variance of P&L

ML based Momentum strategy

Tasks:
    1. Implement a strategy that buys assets when momentum indicators signal strength (e.g., RSI > 70) and sells when they signal weakness (e.g., RSI < 30).
    2. Combine momentum signals with moving averages to enhance the strategy.
    3. Once again try five best combinations of RSI you can find both alone and combined with moving averages
    4. Use a machine learning algorithm with price, and various MA values and RSI as features to see if you can predict buy and sell

Strategy 2: Value-Based Strategies

Objectives:
• Understand fundamental metrics such as P/E ratios, book value
• Backtest value-based strategies using historical data.
• Evaluate the performance of the strategies.

Tasks:
    1. Buy stocks with low P/E ratio compared to historical average
    2. The Price-to-Book ratio compares a company's market value to its book value (the net asset value on the balance sheet). A low P/B ratio may indicate that the stock is undervalued relative to its assets.
    3. Use a machine learning algorithm with price, P/E and PtB values as features to see if you can predict buy and sell

Strategy 3: Sentiment-Based Strategies.

▪ Understand and implement sentiment-based trading strategies.
▪ Analyze sentiment data from news articles, social media, and other sources.
▪ Backtest sentiment-based strategies using historical sentiment data.
▪ Evaluate the performance of sentiment-based strategies.

Implement a Sentiment-Based Trading Strategy:
    ▪ Develop a strategy that buys stocks with positive sentiment and sells stocks with negative sentiment.
    ▪ Experiment with different thresholds for sentiment scores to refine the strategy.
    ▪ Combine sentiment analysis with the moving average and value-based strategies from previous sections.
    ▪ Explore how sentiment signals can enhance or detract from other strategies.
    ▪ Use historical sentiment data alongside market data to backtest the sentiment-based strategy.